In [ ]:
import io, os, sys, collections
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = 1 << 30
class MCMF:
  def __init__(self, V, src, snk) :
    self.len = V
    self.G = [[] for _ in range(V)]
    self.C = [[0] * V for _ in range(V)]
    self.F = [[0] * V for _ in range(V)]
    self.W = [[0] * V for _ in range(V)]
    self.chk = [False] * V
    self.vis = [False] * V
    self.lvl = [0] * V
    self.src = src
    self.snk = snk
    self.mc = 0

  def addEdge(self, u, v, c, w) :
    self.G[u].append(v)
    self.G[v].append(u)
    self.C[u][v] = c
    self.W[u][v] = w
    self.W[v][u] = -w

  def bfs(self) :
    Q = collections.deque([self.src])
    inQ = [False] * self.len 
    lvl = [INF] * self.len
    lvl[self.src] = 0

    while Q :
      u = Q.popleft()
      inQ[u] = False

      for v in self.G[u] :
        if self.C[u][v] > self.F[u][v] and lvl[v] > lvl[u] + self.W[u][v] :
          lvl[v] = lvl[u] + self.W[u][v]
          if not inQ[v] :
            Q.append(v)
            inQ[v] = True
    
    self.lvl = lvl
    return self.lvl[self.snk] != INF

  def dfs(self, u, w) :
    if u == self.snk : return w
    flow = w
    self.vis[u] = True

    for v in self.G[u] :
      if not flow : break
      if not self.vis[v] and not self.chk[v] and self.C[u][v] - self.F[u][v] > 0 and self.lvl[v] == self.lvl[u] + self.W[u][v] :
        res = self.dfs(v, min(flow, self.C[u][v] - self.F[u][v]))
        if res == 0 : 
          self.chk[v] = True
          continue
        self.F[u][v] += res
        self.F[v][u] -= res
        self.mc += res * self.W[u][v]
        flow -= res
    
    self.vis[u] = False
    return w - flow

  def calc(self) :
    mf = 0
    while self.bfs() :
      self.chk = [False] * self.len
      self.vis = [False] * self.len
      mf += self.dfs(self.src, INF)
    return self.mc, mf

def sol() :
  N, M = map(int, input().split()) #사람, 서점
  V = N + M + 2
  src, snk = V - 2, V - 1
  mcmf = MCMF(V, src, snk)

  man = list(map(int, input().split()))
  lib = list(map(int, input().split()))

  for i, m in enumerate(man) :
    mcmf.addEdge(src, i, m, 0)
  for i, l in enumerate(lib) :
    mcmf.addEdge(i + N, snk, l, 0)

  quantity = [[*map(int, input().split())] for _ in range(M)]
  cost = [[*map(int, input().split())] for _ in range(M)]

  for i, (Q, C) in enumerate(zip(quantity, cost)) :
    for j, (q, c) in enumerate(zip(Q, C)) :
      mcmf.addEdge(j, i+N, q, c)
  
  mc, mf = mcmf.calc()
  sys.stdout.write(f'{mf}\n{mc}')
  
sol()

### 자력솔
- 11405 + 11406, 말그대로 최소비용 최대유량을 입력대로 처리하면 된다.